## Disclaimer and license
Copyright 2021 Google LLC. This solution, including any related sample code or data, is made available on an “as is,” “as available,” and “with all faults” basis, solely for illustrative purposes, and without warranty or representation of any kind. This solution is experimental, unsupported and provided solely for your convenience. Your use of it is subject to your agreements with Google, as applicable, and may constitute a beta feature as defined under those agreements. To the extent that you make any data available to Google in connection with your use of the solution, you represent and warrant that you have all necessary and appropriate rights, consents and permissions to permit Google to use and process that data. By using any portion of this solution, you acknowledge, assume and accept all risks, known and unknown, associated with its usage, including with respect to your deployment of any portion of this solution in your systems, or usage in connection with your business, if at all.

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# 5.Model Evaluation and Diagnostics

This notebook demonstrates the evaluation of a LTV regression model by using the
[Regression Diagnostics](https://github.com/google/gps_building_blocks/blob/master/py/gps_building_blocks/ml/diagnostics/regression.py)
module.

This evaluation consists of:
* Model performance with respect to a variety of metrics.
* Plots to understand the model performance to design media experiments.
* Model insights (the relationship between features and the prediction values) helping to generate new business insights.
* Insights helping to diagnose the model to make sure it is reasonable.

## Requirements
The model and the testing dataset should be available in GCP BigQuery.

## Install and import required modules

In [ ]:
# Uncomment to install required python modules
#!pip install -r requirements.txt --quiet

In [ ]:
from gps_building_blocks.cloud.utils import bigquery as bigquery_utils
from gps_building_blocks.ml.diagnostics import regression

## Set parameters

In [ ]:
# GCP Project ID
PROJECT_ID = 'project_id'

# BigQuery dataset name containing the testing predictions dataset
PREDICTION_DATASET = 'predictions_dataset'

# BigQuery table name containing the testing predictions dataset
PREDICTION_TABLE = 'predictions_table'

# Name of the column in the prediction table with the predicted label
PREDICTED_LABEL = 'predicted_label'

# Name of the column in the prediction table with the actual label
ACTUAL_LABEL = 'actual_label'

In [ ]:
bq_utils = bigquery_utils.BigQueryUtils(project_id=PROJECT_ID)

## Read test dataset

In this step, we assume the test dataset containing both prediction and actual values is available in BigQuery.

In [ ]:
sql = f"""
  SELECT *
  `{PROJECT_ID}.{PREDICTION_DATASET}.{PREDICTION_TABLE}`;
"""

df_pred_test = bq_utils.run_query(prediction_query).to_dataframe()
df_pred_test.head()

## Run regression diagnostics

### Calculate performance metrics

To check the regression model performance, we calculated the following metrics for diagnostic:
- Mean squared error: a risk metric corresponding to the expected value of the squared error, calculated by taking the mean of squared error.
- Root mean squared error: the root value of the mean squared error.
- Mean absolute error: a risk metric corresponding to the expected of the absolute value, calculated by taking the mean of absolute error.
- Mean absolute percentage error: an evaluation metric for regression problems, sensitive to relative errors, calculated by taking the mean of the absolute percentage error.
- R-squared: coefficient of determination, representing the proportion of variance that has been explained by the independent variables in the model.
- Pearson correlation: a correlation metric between actual and predicted labels.

In [ ]:
perf_metrics = regression.calc_performance_metrics(
    labels=df_pred_test[ACTUAL_LABEL].VALUES,
    predictions=df_pred_test[PREDICTED_LABEL].values,
    deciamal_points=4)

print(perf_metrics)

### Scatter Plots of prediction values and residuals

The following function plots:
1. the scatter plot of actual values versus prediction values
2. the scatter plot of residuals versus predicted values
From the first plot, we can see how prediction values differ from actual values, whether there exists strong correlation between the predictions and actual results. In the second plot, the residual plot, we can see how the residuals deviate from the line at zero since the residual equals to the actual value minus the prediction value. Ideal residual plots would be symmetrically distributed and cluster close to y=0 value.

In [ ]:
regression.plot_prediction_residuals(
    labels=df_pred_test[ACTUAL_LABEL].VALUES,
    predictions=df_pred_test[PREDICTED_LABEL].values)

### Calculate performance metrics for the bins of the prediction values

The following function calculates performance metrics for each bin of the predictions. The default number of the bins of the prediction values is a parameter and here we use 10 as an example. The calculation is firstly rankd the prediction values from the highest to the lowest and then devide the predictions into 10 bins such that the first bin contains the highest 10% of the predictions and the second bin contains the next 10% of the predictions and so on. The following metrics are calculated for each bin:
- mean_label: Mean of actual values in the bin.
- mean_prediction: Mean of predictions in the bin.
- rmse: Root mean squared error.
- mape: Mean absolute percentage error.
- corr: Pearson correlation coefficient.

In [ ]:
bin_metrics = regression.calc_reg_bin_metrics(
    labels=df_pred_test[ACTUAL_LABEL].VALUES,
    predictions=df_pred_test[PREDICTED_LABEL].values,
    number_bins=10)

bin_metrics

### Plot performance metrics for the bins of the prediction values

The following function plots performance metrics in each bin using bin_metrics calculated in last section. These plots allow us to have better understanding of predictions in the test dataset. In the first subplot, it shows the mean of actual and prediction values in each bin. An ideal plot would be the height of bars of both prediction and actual values decrease as the number of the bin increases. In the rest three plots (MAPE, RMSE, corr), the evaluation metric has same interpretation as in the regression models.

In [ ]:
regression.plot_reg_bin_metrics(
    bin_metrics=bin_metrics,
    fig_width=25,
    fig_height=30)

### Plot actual distribution over the bins of the prediction values

In [ ]:
# TODO(): Add functions to plot actual distribution over the bins of the prediction values.

### Confusion matrix of the actual vs predicted bins

In [ ]:
# TODO(): Add functions to calculate and plot confusion matrix.